# **Imports and Downloads**

In [1]:
!pip uninstall -y numpy
!pip install numpy==1.26.4 --upgrade --force-reinstall --quiet
!pip install --upgrade pennylane pennylane-lightning

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 48.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━

In [ ]:
import os
os.kill(os.getpid(), 9)

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)

Mounted at /content/drive


In [2]:
# Core libraries
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset

import numpy as np
import pandas as pd
import pickle
import math
import re
import os
import random
import time
import matplotlib.pyplot as plt
from collections import Counter

# Quantum computing
import pennylane as qml
from pennylane import numpy as pnp
from pennylane.optimize import AdamOptimizer

# NLP and preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.util import ngrams

# Sklearn tools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
pnp.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# **Data Loading and Preprocessing**

In [3]:
with open("/content/drive/MyDrive/QML-Research/Data/sentiment labelled sentences/amazon_cells_labelled.txt", "r") as f:
    lines = f.readlines()

sentences = [line.split("\t")[0] for line in lines]
labels = [int(line.split("\t")[1]) for line in lines]

In [4]:
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))
domain_neutral_words = {
    "phone", "product", "battery", "headset", "quality", "one", "use"
}
stop_words.update(domain_neutral_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
def clean_and_tokenize(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"[^a-z0-9\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

cleaned_sents = [clean_and_tokenize(sentence) for sentence in sentences]

In [6]:
max_len = 10
for i in range(len(cleaned_sents)):
  if (len(cleaned_sents[i]) < max_len):
    cleaned_sents[i] += ["<PAD>"] * (max_len - len(cleaned_sents[i]))
  else:
    cleaned_sents[i] = cleaned_sents[i][:max_len]

# **GloVE Word Embeddings**

In [7]:
def load_glove_embeddings(file_path):
    embeddings = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.strip().split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

In [8]:
glove_path = '/content/drive/MyDrive/QML-Research/Data/glove.6B.100d.txt'
glove = load_glove_embeddings(glove_path)

# **AutoEncoder**

In [9]:
class GloVeAutoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(GloVeAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 64),
            nn.ReLU(),
            nn.Linear(64, input_dim)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [10]:
all_words = list(glove.keys())
all_vectors = np.array([glove[word] for word in all_words])
all_vectors = normalize(all_vectors)
word_tensor = torch.tensor(all_vectors).float()

In [11]:
latent_dim = 8
epochs = 100
save_path = '/content/drive/MyDrive/QML-Research/Autoencoder/glove_autoencoder_normalized_8.pth'
# save_path = '/content/drive/MyDrive/QML-Research/Autoencoder/glove_autoencoder_normalized_32.pth'

In [12]:
if os.path.exists(save_path):
    print(f"Loading Autoencoder from {save_path}")
    autoencoder = GloVeAutoencoder(input_dim=100, latent_dim=latent_dim)
    autoencoder.load_state_dict(torch.load(save_path, map_location=torch.device('cpu')))
else:
    print("Training Autoencoder")
    autoencoder = GloVeAutoencoder(input_dim=100, latent_dim=latent_dim)
    optimizer = torch.optim.Adam(autoencoder.parameters(), lr=1e-3)
    criterion = nn.MSELoss()

    for epoch in range(epochs):
        optimizer.zero_grad()
        reconstructed = autoencoder(word_tensor)
        loss = criterion(reconstructed, word_tensor)
        loss.backward()
        optimizer.step()

        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.6f}")

    torch.save(autoencoder.state_dict(), save_path)
    print(f"Saved Autoencoder serialized model in drive @ {save_path}")

Loading Autoencoder from /content/drive/MyDrive/QML-Research/Autoencoder/glove_autoencoder_normalized_8.pth


In [13]:
autoencoder.eval()
with torch.no_grad():
    compressed_vectors = autoencoder.encoder(word_tensor).numpy()

reduced_embeddings = {
    word: compressed_vectors[i]
    for i, word in enumerate(all_words)
}

# **Embedding**

In [14]:
def sentence_to_vec(sentence, embeddings, dim):
    vectors = []
    for word in sentence:
        if word in embeddings:
            vectors.append(embeddings[word])
        else:
            vectors.append(np.zeros(dim))
    return vectors

def embed_sentences(cleaned_sents, embeddings, dim):
    return np.array([sentence_to_vec(tokens, embeddings, dim) for tokens in cleaned_sents])

In [15]:
X_embed_np = embed_sentences(cleaned_sents, reduced_embeddings, dim=8)
X_embed_np = (X_embed_np - X_embed_np.min()) * (np.pi / (X_embed_np.max() - X_embed_np.min()))

X_embed = torch.tensor(X_embed_np).long()
y_embed = torch.tensor(labels).long()

# **Dataset and DataLoader**

In [16]:
class AmazonDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    X_embed, y_embed, test_size=0.2, stratify=y_embed.numpy(), random_state=42
)

train_dataset = AmazonDataset(X_train, y_train)
test_dataset = AmazonDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1)

# **Utility Functions**

In [18]:
def softmax(logits):
    logits = qml.numpy.array(logits)
    logits = logits - qml.numpy.max(logits)
    exps = qml.numpy.exp(logits)
    return exps / qml.numpy.sum(exps)

def cross_entropy(label, probs):
    p = qml.numpy.clip(probs[label], 1e-10, 1.0)
    return -qml.numpy.log(p)

# def reshape_params(flat_params):
#     wpw = flat_params[:80].reshape((10, 8))
#     clf = flat_params[80:].reshape((8,))
#     return wpw, clf

# **QRNN Model Layers**

In [19]:
"""
First Variation of QRNN that showed learning during training, Achieving a training accuracy of 67% (Highest)
and a peak testing accuracy of 55.5% with a training duration of 50-60 epochs. The layers have been split
into Unitary and entanglement and have been implemnted as functions. This model processes the whole sequence
(All Timesteps) at once in contrast to the previous iterations that processes one time-step/word at a time.
"""

# def dense_angle_embedding(x, wires):
#     # For 32 dim:
#     # chunk_size = 8
#     chunk_size = 2

#     for i, wire in enumerate(wires):
#         for j in range(chunk_size):
#             idx = i * chunk_size + j
#             qml.RX(x[idx], wires=wire)
#             qml.RY(x[idx], wires=wire)
#             qml.RZ(x[idx], wires=wire)

# def variational_block(weights, wires):
#     for i, wire in enumerate(wires):
#         qml.RY(weights[i], wires=wire)
#         qml.RZ(weights[i + len(wires)], wires=wire)

# def entangle(wires):
#     for i in range(len(wires) - 1):
#         qml.CNOT(wires=[wires[i], wires[i + 1]])
#     qml.CNOT(wires=[wires[-1], wires[0]])

'\nFirst Variation of QRNN that showed learning during training, Achieving a training accuracy of 67% (Highest)\nand a peak testing accuracy of 55.5% with a training duration of 50-60 epochs. The layers have been split\ninto Unitary and entanglement and have been implemnted as functions. This model processes the whole sequence\n(All Timesteps) at once in contrast to the previous iterations that processes one time-step/word at a time.\n'

In [20]:
"""
The second iteration of the previous model introducing major architectural changes (All layers are implemented as functions):
a. Stacked QRNN (2 layered QRNN), with each QRB having seperate weights (No weight sharing)
b. Interaction layers implemented seperate from the QRBs (Quantum Recurrent Block)
c. Introduced an additional interaction layer (classification layer) before quantum measurement
"""

def input_layer(x_sequence, wires):
    for word_vec in x_sequence:
        for i, wire in enumerate(wires):
            qml.RX(word_vec[2 * i], wires=wire)
            qml.RY(word_vec[2 * i + 1], wires=wire)
            qml.RZ(word_vec[2 * i], wires=wire)

# def qrnn_layer(weights, wires):
#     for i, wire in enumerate(wires):
#         qml.RY(weights[i], wires=wire)
#         qml.RZ(weights[i + len(wires)], wires=wire)

def qrnn_layer(weights, wires):
    for i, wire in enumerate(wires):
        qml.Rot(weights[i, 0], weights[i, 1], weights[i, 2], wires=wire)

def interaction_layer(wires):
    for i in range(len(wires)):
        qml.CNOT(wires=[wires[i], wires[(i + 1) % len(wires)]])

# def final_interaction_layer(weights, wires):
#     for i, wire in enumerate(wires):
#         qml.RY(weights[i], wires=wire)
#         qml.RZ(weights[i + len(wires)], wires=wire)

def final_interaction_layer(weights, wires):
    for i, wire in enumerate(wires):
        qml.Rot(*weights[i], wires=wire)

# **Model**

In [21]:
# def forward(input_sequence, weights_in, weights_h, hidden_init=None):
#     """
#     input_sequence: (10, 32) numpy array
#     weights_in: (6,) numpy array
#     weights_h: (2,) numpy array
#     hidden_init: optional scalar float, default is small random value
#     """
#     if hidden_init is None:
#         hidden = np.random.uniform(-0.1, 0.1)
#     else:
#         hidden = hidden_init

#     for word_vec in input_sequence:
#         logits = quantum_step(word_vec, hidden, weights_in, weights_h)
#         hidden = np.tanh(logits[0] + logits[1])

#     return logits

In [22]:
def qrnn_model(x_sequence, flat_params, wires):
    # w_qrnn1 = flat_params[:8]
    # w_qrnn2 = flat_params[8:16]
    # w_classifier = flat_params[16:32]

    w_qrb1 = flat_params[:12].reshape((4, 3))
    w_qrb2 = flat_params[12:24].reshape((4, 3))
    # w_classifier = flat_params[24:]
    w_classifier = flat_params[24:].reshape((4, 3))

    input_layer(x_sequence, wires)
    qrnn_layer(w_qrb1, wires)
    interaction_layer(wires)

    qrnn_layer(w_qrb2, wires)
    interaction_layer(wires)

    final_interaction_layer(w_classifier, wires)

# **QNode**

In [23]:
n_qubits = 4
# dev = qml.device("default.qubit", wires=n_qubits)
dev = qml.device("lightning.qubit", wires=n_qubits)

In [24]:
"""
Simple Qnode with Dense Angle Encoding for the 8 dimensional inputs,
A simple Ring entanglement and parameteried training of input weights
and hidden weights. Returns 2 quantum measurables.
"""

# @qml.qnode(dev, interface="torch")
# def quantum_step(inputs, hidden, weights_in, weights_h):
#     #Encoding
#     dense_angle_embedding(inputs, wires=[0, 1, 2, 3])
#     qml.RY(hidden, wires=3)

#     # Interaction Layer/Entanglement
#     qml.CNOT(wires=[0, 1])
#     qml.CNOT(wires=[1, 2])
#     qml.CNOT(wires=[2, 3])
#     qml.CNOT(wires=[3, 0])

#     # Parametrized trainable unitaries on input wires
#     for i in range(3):
#         qml.RY(weights_in[i], wires=i)
#         qml.RZ(weights_in[i + 3], wires=i)

#     # Parametrized unitaries on hidden wire
#     qml.RY(weights_h[0], wires=3)
#     qml.RZ(weights_h[1], wires=3)

#     return qml.expval(qml.PauliZ(2)), qml.expval(qml.PauliZ(3))

'\nSimple Qnode with Dense Angle Encoding for the 8 dimensional inputs,\nA simple Ring entanglement and parameteried training of input weights\nand hidden weights. Returns 2 quantum measurables.\n'

In [25]:
"""
Qnode to process a whole sequence at a time in contrast to the initial attempts at
processing a time step at once. It still uses 4 qubit dense angle encoding, but also
preserves the the hidden wire's quantum state throughout the sequence processing,
which helps avoid re-encoding the hidden state at each time-step.
"""

# @qml.qnode(dev, interface="autograd")
# def full_sentence_qnode(sentence_embeds, weights_per_word, classifier_weights):
#     wires = list(range(n_qubits))

#     # Processes a whole sequence (10 words * 8 Dimensional vector embedding)
#     for t in range(len(sentence_embeds)):
#         dense_angle_embedding(sentence_embeds[t], wires)
#         entangle(wires)
#         variational_block(weights_per_word[t], wires)

#     # Final classifier
#     entangle(wires)
#     variational_block(classifier_weights, wires)

#     return qml.expval(qml.PauliZ(wires[2])), qml.expval(qml.PauliZ(wires[3]))

"\nQnode to process a whole sequence at a time in contrast to the initial attempts at\nprocessing a time step at once. It still uses 4 qubit dense angle encoding, but also\npreserves the the hidden wire's quantum state throughout the sequence processing,\nwhich helps avoid re-encoding the hidden state at each time-step.\n"

In [26]:
@qml.qnode(dev, interface="autograd")
def stacked_qrnn(x_sequence, flat_params):
    wires = list(range(n_qubits))
    qrnn_model(x_sequence, flat_params, wires)
    return qml.expval(qml.PauliZ(wires[2])), qml.expval(qml.PauliZ(wires[3]))

# **Parameter Initialization**

In [27]:
# weights_in = np.random.uniform(0, 2 * np.pi, size=6)
# weights_h  = np.random.uniform(0, 2 * np.pi, size=2)
# optimizer = AdamOptimizer(stepsize=0.01)
# epochs = 20

In [28]:
# weights_per_word = qml.numpy.array(np.random.uniform(0, 2*np.pi, (10, 8)), requires_grad=True)
# classifier_weights = qml.numpy.array(np.random.uniform(0, 2*np.pi, (8,)), requires_grad=True)

# flat_params = qml.numpy.concatenate([
#     weights_per_word.flatten(),
#     classifier_weights.flatten()
# ])

# optimizer = AdamOptimizer(stepsize=0.01)
# epochs = 60

In [29]:
flat_params = qml.numpy.array(
    # np.random.uniform(0, 2 * np.pi, size=(32,)),
    # np.random.normal(0, 0.01, size=(32,)),
    np.random.normal(0, 0.01, size=(36,)),
    requires_grad=True
)

optimizer = AdamOptimizer(stepsize=0.005)
epochs = 80

# **Directory Creation**

In [ ]:
base_dir = "/content/drive/MyDrive/QML-Research/Analysis"

folders_to_create = [
    "logs/qrnn_outputs",
    "plots/loss_trend",
    "plots/accuracy_trend",
    "plots/test_boxplots",
]

for folder in folders_to_create:
    path = os.path.join(base_dir, folder)
    os.makedirs(path, exist_ok=True)
    print(f"Created (or already exists): {path}")

Created (or already exists): /content/drive/MyDrive/QML-Research/Analysis/logs/qrnn_outputs
Created (or already exists): /content/drive/MyDrive/QML-Research/Analysis/plots/loss_trend
Created (or already exists): /content/drive/MyDrive/QML-Research/Analysis/plots/accuracy_trend
Created (or already exists): /content/drive/MyDrive/QML-Research/Analysis/plots/test_boxplots


# **Training**

In [30]:
train_losses = []
train_accuracies = []

print("Training Loop\n")
start_time = time.time()

for epoch in range(1, epochs + 1):
    epoch_loss = 0
    correct = 0
    total = 0
    epoch_start = time.time()

    for xb, yb in train_loader:
        x_np = xb.squeeze(0).numpy()
        y_np = yb.item()
        x_qml = qml.numpy.array(x_np, requires_grad=False)

        def cost(flat_params):
          # wpw, clf = reshape_params(flat_params)
          # logits = full_sentence_qnode(x_qml, wpw, clf)
          logits = stacked_qrnn(x_qml, flat_params)
          logits = qml.numpy.array(logits)
          probs = softmax(logits)
          return cross_entropy(y_np, probs)



        # weights_in, weights_h = optimizer.step(cost, (weights_in, weights_h))
        # logits = forward(x_np, weights_in, weights_h)
        flat_params = optimizer.step(cost, flat_params)
        # weights_per_word, classifier_weights = reshape_params(flat_params)
        # logits = full_sentence_qnode(x_qml, weights_per_word, classifier_weights)
        logits = stacked_qrnn(x_qml, flat_params)
        probs = softmax(logits)
        loss = cross_entropy(y_np, probs)

        pred_label = qml.numpy.argmax(probs)

        if pred_label == y_np:
            correct += 1
        epoch_loss += loss
        total += 1

    avg_loss = epoch_loss / total
    acc = correct / total

    train_losses.append(avg_loss)
    train_accuracies.append(acc)

    epoch_time = time.time() - epoch_start
    print(f"Epoch {epoch:02d} | Loss: {avg_loss:.4f} | Acc: {acc:.4f} | Time: {epoch_time:.2f}s")

total_time = time.time() - start_time
print(f"\nTraining complete in {total_time:.2f} seconds.")

Training Loop

Epoch 01 | Loss: 0.7178 | Acc: 0.5062 | Time: 48.89s
Epoch 02 | Loss: 0.6904 | Acc: 0.5500 | Time: 46.75s
Epoch 03 | Loss: 0.6821 | Acc: 0.5563 | Time: 46.36s
Epoch 04 | Loss: 0.6779 | Acc: 0.5713 | Time: 46.74s
Epoch 05 | Loss: 0.6744 | Acc: 0.5900 | Time: 46.57s
Epoch 06 | Loss: 0.6708 | Acc: 0.6038 | Time: 46.28s
Epoch 07 | Loss: 0.6703 | Acc: 0.6012 | Time: 50.52s
Epoch 08 | Loss: 0.6693 | Acc: 0.5962 | Time: 49.04s
Epoch 09 | Loss: 0.6671 | Acc: 0.6075 | Time: 48.55s
Epoch 10 | Loss: 0.6674 | Acc: 0.6062 | Time: 47.12s
Epoch 11 | Loss: 0.6662 | Acc: 0.6112 | Time: 47.28s
Epoch 12 | Loss: 0.6672 | Acc: 0.6088 | Time: 47.22s
Epoch 13 | Loss: 0.6671 | Acc: 0.6150 | Time: 47.74s
Epoch 14 | Loss: 0.6672 | Acc: 0.6025 | Time: 47.49s
Epoch 15 | Loss: 0.6679 | Acc: 0.6138 | Time: 46.87s
Epoch 16 | Loss: 0.6664 | Acc: 0.6150 | Time: 47.58s
Epoch 17 | Loss: 0.6658 | Acc: 0.6112 | Time: 47.20s
Epoch 18 | Loss: 0.6670 | Acc: 0.6075 | Time: 47.15s
Epoch 19 | Loss: 0.6666 | Acc: 

# **Testing**

In [31]:
# def evaluate_test(weights_in, weights_h):
# def evaluate_test(weights_per_word, classifier_weights):
def evaluate_test(flat_params):
    correct = 0
    total = 0
    test_loss = 0

    for xb, yb in test_loader:
        x_np = xb.squeeze(0).numpy()
        y_np = yb.item()
        x_qml = qml.numpy.array(x_np, requires_grad=False)

        # logits = full_sentence_qnode(x_qml, weights_per_word, classifier_weights)
        logits = stacked_qrnn(x_qml, flat_params)
        probs = softmax(logits)
        loss = cross_entropy(y_np, probs)

        pred = qml.numpy.argmax(probs)

        if pred == y_np:
            correct += 1
        test_loss += loss
        total += 1

    return test_loss / total, correct / total

# **Logging**

In [32]:
def log_test_results(model_name, test_acc, test_loss):
    log_path = "/content/drive/MyDrive/QML-Research/Analysis/logs/qrnn_outputs/accuracy_logs.txt"

    with open(log_path, "a") as f:
        f.write(f"[Model: {model_name}]\n")
        f.write(f"Test Accuracy: {test_acc:.4f}\n")
        f.write(f"Test Loss: {test_loss:.4f}\n\n")

    print(f"Logged test results to: {log_path}")

In [33]:
def save_training_plots(model_name, train_losses, train_accuracies):
    base_path = "/content/drive/MyDrive/QML-Research/Analysis/plots"

    # Loss Plot
    plt.figure(figsize=(8, 5))
    plt.plot(train_losses, label="Training Loss")
    plt.title(f"{model_name} - Loss Trend")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{base_path}/loss_trend/loss_plot_{model_name}.png")
    plt.close()

    # Accuracy Plot
    plt.figure(figsize=(8, 5))
    plt.plot(train_accuracies, label="Training Accuracy", color='green')
    plt.title(f"{model_name} - Accuracy Trend")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{base_path}/accuracy_trend/acc_plot_{model_name}.png")
    plt.close()

    print(f"Plots saved to: {base_path}/loss_trend/ and /accuracy_trend/")

In [34]:
# test_loss, test_acc = evaluate_test(weights_in, weights_h)
# test_loss, test_acc = evaluate_test(weights_per_word, classifier_weights)
test_loss, test_acc = evaluate_test(flat_params)
log_test_results("Stacked_QRNN_v6", test_acc, test_loss)
save_training_plots("Stacked_QRNN_v6", train_losses, train_accuracies)

Logged test results to: /content/drive/MyDrive/QML-Research/Analysis/logs/qrnn_outputs/accuracy_logs.txt
Plots saved to: /content/drive/MyDrive/QML-Research/Analysis/plots/loss_trend/ and /accuracy_trend/


# **Model Saving and Loading**

In [35]:
with open('/content/drive/MyDrive/QML-Research/Model Saves/stacked_qrnn_v6_params.pkl', 'wb') as f:
    pickle.dump(flat_params, f)

print("Saved Model parameters")

Saved Model parameters


In [ ]:
with open('/content/drive/MyDrive/QML-Research/Models/stacked_qrnn_v6_params.pkl', 'rb') as f:
    flat_params = pickle.load(f)

weights_per_word, classifier_weights = reshape_params(flat_params)
print("Model parameters loaded")